# WA Mining Operations Data Generator
## Western Australia Mining Performance Dashboard - Project #3

**Author:** Flavio Coltellacci  
**Date:** January 2026  
**Purpose:** Generate realistic synthetic dataset for mining operations BI dashboard

---

## Overview

This notebook generates a synthetic dataset representing 12 months (2025) of mining operations data across 4 Western Australian mine sites.

**Data Generated:**
1. **mine_sites.csv** - Master data for 5 mine sites (4 operational + 1 care & maintenance)
2. **daily_production.csv** - 1,825 daily production records (365 days × 5 sites)
3. **equipment_performance.csv** - ~8,400 equipment records (365 days × 23 units)
4. **safety_incidents.csv** - ~360 safety incidents (realistic frequency)
5. **daily_costs.csv** - 1,825 cost records (by site and date)
6. **safety_metrics_summary.csv** - TRIFR/LTIFR calculations by site

**Industry Research:**
All parameters based on 2025 Western Australian mining industry benchmarks:
- Department of Mines, Industry Regulation and Safety (DMIRS) WA
- WorkSafe WA statistics
- Major mining companies' operational reports

**Key Features:**
- Realistic operational variance (±5-10% from targets)
- Seasonal impacts (Jan-Mar cyclone season: 15-20% production loss in Pilbara)
- Quarterly maintenance shutdowns (2-3 days)
- Random equipment failures based on MTBF
- Industry-accurate KPIs: TRIFR, OEE, AISC, recovery rates

---

## Technologies Used
- Python 3.10+
- Pandas (data manipulation)
- NumPy (random generation)
- Datetime (date handling)

---

## How to Use
1. Run all cells sequentially
2. Download generated CSV files from Colab
3. Import into SQLite database
4. Use for analysis and visualization

---

In [ ]:
# =====================================================
# WA MINING OPERATIONS DATA GENERATOR
# Project: Mining Operations Performance Dashboard
# Date: December 2025
# =====================================================

# Install required libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

print("=" * 60)
print("WA MINING OPERATIONS DATA GENERATOR")
print("Generating realistic 2025 mining data for 5 WA mine sites")
print("=" * 60)

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# =====================================================
# DATASET 1: MINE SITES MASTER DATA
# =====================================================

print("\n[1/5] Generating Mine Sites Master Data...")

mine_sites = {
    'site_id': ['IRON001', 'GOLD001', 'LITH001', 'COPP001', 'NICK001'],
    'site_name': [
        'Iron Ridge Mine',
        'Golden Valley Mine',
        'Lithium Creek Mine',
        'Copper Hills Mine',
        'Nickel Point Mine'
    ],
    'commodity': ['Iron Ore', 'Gold', 'Lithium', 'Copper', 'Nickel'],
    'region': ['Pilbara', 'Goldfields', 'Pilbara', 'Mid West', 'Kambalda'],
    'latitude': [-22.5847, -30.7458, -21.8956, -28.4521, -31.2089],
    'longitude': [117.8853, 121.4656, 118.7234, 117.1234, 121.6447],
    'mine_type': ['Open Pit', 'Open Pit + Underground', 'Open Pit', 'Underground', 'Underground'],
    'status': ['Operational', 'Operational', 'Operational', 'Operational', 'Care & Maintenance'],
    'annual_target_tonnes': [50000000, 13000000, 700000, 20000, 0],  # Iron in tonnes, Gold in tonnes ore, Lithium in concentrate, Copper in metal
    'daily_target_tonnes': [137000, 35600, 1920, 55, 0],
    'target_recovery_rate': [98.0, 91.0, 70.0, 88.0, 0.0],  # %
    'target_head_grade': [62.0, 1.2, 1.3, 2.8, 0.0],  # Fe%, g/t Au, Li2O%, Cu%
    'workforce_count': [850, 1200, 450, 280, 25],  # 25 for care & maintenance
    'automation_level': ['High', 'Medium', 'Medium', 'Low', 'N/A'],  # Autonomous equipment %
    'cost_per_unit_target': [27.0, 2500.0, 640.0, 1.85, 0.0],  # Iron $/t, Gold $/oz, Lithium $/t, Copper $/lb (in AUD)
    'operating_since': ['2023-06-01', '1989-03-15', '2021-11-01', '2007-04-20', '2018-02-10']
}

df_sites = pd.DataFrame(mine_sites)

print(f"✓ Created mine sites data: {len(df_sites)} sites")
print(df_sites[['site_id', 'site_name', 'commodity', 'status']].to_string(index=False))

# =====================================================
# DATASET 2: DAILY PRODUCTION DATA (365 days × 5 sites)
# =====================================================

print("\n[2/5] Generating Daily Production Data (365 days)...")

start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

production_records = []

for site in df_sites.itertuples():
    site_id = site.site_id
    site_name = site.site_name
    commodity = site.commodity
    daily_target = site.daily_target_tonnes
    target_grade = site.target_head_grade
    target_recovery = site.target_recovery_rate
    status = site.status

    for date in date_range:
        # Skip production if site is in Care & Maintenance
        if status == 'Care & Maintenance':
            production_records.append({
                'date': date.strftime('%Y-%m-%d'),
                'site_id': site_id,
                'site_name': site_name,
                'commodity': commodity,
                'status': 'Care & Maintenance',
                'tonnes_mined': 0,
                'tonnes_processed': 0,
                'head_grade': 0.0,
                'recovery_rate': 0.0,
                'metal_produced': 0.0,
                'plan_adherence_pct': 0.0,
                'shift_1_tonnes': 0,
                'shift_2_tonnes': 0,
                'operational_hours': 0,
                'downtime_hours': 24.0,
                'weather_delay_hours': 0.0
            })
            continue

        # Seasonal factors (cyclones Jan-Mar for Pilbara)
        is_cyclone_season = (date.month in [1, 2, 3]) and (site.region == 'Pilbara')
        cyclone_impact = np.random.choice([0, 0, 0, 0.3, 0.5, 0.8], p=[0.85, 0.10, 0.03, 0.01, 0.005, 0.005]) if is_cyclone_season else 0

        # Maintenance shutdowns (quarterly, 2-3 days)
        is_maintenance = (date.day in [15, 16]) and (date.month in [3, 6, 9, 12])
        maintenance_impact = 0.7 if is_maintenance else 0

        # Base production efficiency (realistic variance)
        base_efficiency = np.random.normal(0.95, 0.08)  # Mean 95%, std 8%
        base_efficiency = np.clip(base_efficiency, 0.75, 1.10)  # Cap at 75%-110%

        # Total impact
        total_impact = max(cyclone_impact, maintenance_impact)
        actual_efficiency = base_efficiency * (1 - total_impact)

        # Calculate tonnes
        tonnes_mined = int(daily_target * actual_efficiency * np.random.uniform(0.95, 1.05))
        tonnes_processed = int(tonnes_mined * 0.98)  # 2% loss in handling

        # Grade variability (±5% from target)
        head_grade = target_grade * np.random.uniform(0.95, 1.05)

        # Recovery rate (slightly below target on average)
        recovery_rate = target_recovery * np.random.uniform(0.96, 1.02)
        recovery_rate = np.clip(recovery_rate, target_recovery - 5, target_recovery + 2)

        # Metal produced (varies by commodity)
        if commodity == 'Gold':
            # Gold: tonnes ore × grade (g/t) × recovery × conversion to oz
            metal_produced = (tonnes_processed * head_grade * recovery_rate / 100) / 31.1035  # oz
        elif commodity == 'Iron Ore':
            # Iron: just recovered tonnes (no conversion needed)
            metal_produced = tonnes_processed * (recovery_rate / 100)
        elif commodity == 'Lithium':
            # Lithium: concentrate tonnes at SC6 grade
            metal_produced = tonnes_processed * (head_grade / 100) * (recovery_rate / 100)
        elif commodity == 'Copper':
            # Copper: metal tonnes
            metal_produced = tonnes_processed * (head_grade / 100) * (recovery_rate / 100)
        else:
            metal_produced = 0

        # Plan adherence
        plan_adherence = (tonnes_mined / daily_target * 100) if daily_target > 0 else 0
        plan_adherence = np.clip(plan_adherence, 0, 110)

        # Shift breakdown (12-hour shifts)
        shift_split = np.random.uniform(0.45, 0.55)
        shift_1_tonnes = int(tonnes_mined * shift_split)
        shift_2_tonnes = tonnes_mined - shift_1_tonnes

        # Hours
        operational_hours = 24 * (1 - total_impact)
        downtime_hours = 24 - operational_hours
        weather_delay_hours = 24 * cyclone_impact if cyclone_impact > 0 else 0.0

        production_records.append({
            'date': date.strftime('%Y-%m-%d'),
            'site_id': site_id,
            'site_name': site_name,
            'commodity': commodity,
            'status': 'Operational',
            'tonnes_mined': tonnes_mined,
            'tonnes_processed': tonnes_processed,
            'head_grade': round(head_grade, 2),
            'recovery_rate': round(recovery_rate, 1),
            'metal_produced': round(metal_produced, 2),
            'plan_adherence_pct': round(plan_adherence, 1),
            'shift_1_tonnes': shift_1_tonnes,
            'shift_2_tonnes': shift_2_tonnes,
            'operational_hours': round(operational_hours, 1),
            'downtime_hours': round(downtime_hours, 1),
            'weather_delay_hours': round(weather_delay_hours, 1)
        })

df_production = pd.DataFrame(production_records)

print(f"✓ Created daily production data: {len(df_production)} records")
print(f"  Date range: {df_production['date'].min()} to {df_production['date'].max()}")
print(f"  Total tonnes mined: {df_production['tonnes_mined'].sum():,.0f}")

# =====================================================
# DATASET 3: EQUIPMENT PERFORMANCE DATA
# =====================================================

print("\n[3/5] Generating Equipment Performance Data...")

# Equipment fleet definition
equipment_fleet = [
    # Iron Ridge (Autonomous, Pilbara)
    {'site_id': 'IRON001', 'equipment_id': 'TRK-IR-001', 'equipment_type': 'Haul Truck', 'model': 'CAT 793F AHS', 'autonomous': True, 'target_util': 95},
    {'site_id': 'IRON001', 'equipment_id': 'TRK-IR-002', 'equipment_type': 'Haul Truck', 'model': 'CAT 793F AHS', 'autonomous': True, 'target_util': 95},
    {'site_id': 'IRON001', 'equipment_id': 'TRK-IR-003', 'equipment_type': 'Haul Truck', 'model': 'CAT 793F AHS', 'autonomous': True, 'target_util': 95},
    {'site_id': 'IRON001', 'equipment_id': 'EXC-IR-001', 'equipment_type': 'Excavator', 'model': 'Liebherr R9800', 'autonomous': False, 'target_util': 82},
    {'site_id': 'IRON001', 'equipment_id': 'EXC-IR-002', 'equipment_type': 'Excavator', 'model': 'Liebherr R9800', 'autonomous': False, 'target_util': 82},
    {'site_id': 'IRON001', 'equipment_id': 'DOZ-IR-001', 'equipment_type': 'Dozer', 'model': 'CAT D11T', 'autonomous': False, 'target_util': 75},

    # Golden Valley (Manual, Goldfields)
    {'site_id': 'GOLD001', 'equipment_id': 'TRK-GV-001', 'equipment_type': 'Haul Truck', 'model': 'CAT 793F', 'autonomous': False, 'target_util': 80},
    {'site_id': 'GOLD001', 'equipment_id': 'TRK-GV-002', 'equipment_type': 'Haul Truck', 'model': 'CAT 793F', 'autonomous': False, 'target_util': 80},
    {'site_id': 'GOLD001', 'equipment_id': 'TRK-GV-003', 'equipment_type': 'Haul Truck', 'model': 'CAT 793F', 'autonomous': False, 'target_util': 80},
    {'site_id': 'GOLD001', 'equipment_id': 'EXC-GV-001', 'equipment_type': 'Excavator', 'model': 'Liebherr R9800', 'autonomous': False, 'target_util': 82},
    {'site_id': 'GOLD001', 'equipment_id': 'EXC-GV-002', 'equipment_type': 'Excavator', 'model': 'CAT 6060', 'autonomous': False, 'target_util': 82},
    {'site_id': 'GOLD001', 'equipment_id': 'DOZ-GV-001', 'equipment_type': 'Dozer', 'model': 'CAT D11T', 'autonomous': False, 'target_util': 75},
    {'site_id': 'GOLD001', 'equipment_id': 'DOZ-GV-002', 'equipment_type': 'Dozer', 'model': 'CAT D10T2', 'autonomous': False, 'target_util': 75},

    # Lithium Creek (Semi-auto, Pilbara)
    {'site_id': 'LITH001', 'equipment_id': 'TRK-LC-001', 'equipment_type': 'Haul Truck', 'model': 'CAT 793F', 'autonomous': False, 'target_util': 82},
    {'site_id': 'LITH001', 'equipment_id': 'TRK-LC-002', 'equipment_type': 'Haul Truck', 'model': 'CAT 793F', 'autonomous': False, 'target_util': 82},
    {'site_id': 'LITH001', 'equipment_id': 'EXC-LC-001', 'equipment_type': 'Excavator', 'model': 'CAT 6060', 'autonomous': False, 'target_util': 82},
    {'site_id': 'LITH001', 'equipment_id': 'DOZ-LC-001', 'equipment_type': 'Dozer', 'model': 'CAT D10T2', 'autonomous': False, 'target_util': 75},

    # Copper Hills (Underground equipment)
    {'site_id': 'COPP001', 'equipment_id': 'LHD-CH-001', 'equipment_type': 'LHD Loader', 'model': 'CAT R1700', 'autonomous': False, 'target_util': 70},
    {'site_id': 'COPP001', 'equipment_id': 'LHD-CH-002', 'equipment_type': 'LHD Loader', 'model': 'CAT R1700', 'autonomous': False, 'target_util': 70},
    {'site_id': 'COPP001', 'equipment_id': 'LHD-CH-003', 'equipment_type': 'LHD Loader', 'model': 'Sandvik LH517i', 'autonomous': False, 'target_util': 70},
    {'site_id': 'COPP001', 'equipment_id': 'DRL-CH-001', 'equipment_type': 'Jumbo Drill', 'model': 'Sandvik DD421', 'autonomous': False, 'target_util': 45},
    {'site_id': 'COPP001', 'equipment_id': 'DRL-CH-002', 'equipment_type': 'Jumbo Drill', 'model': 'Sandvik DD422i', 'autonomous': False, 'target_util': 45},
]

equipment_records = []

for equip in equipment_fleet:
    site_id = equip['site_id']
    equipment_id = equip['equipment_id']
    equipment_type = equip['equipment_type']
    model = equip['model']
    is_autonomous = equip['autonomous']
    target_util = equip['target_util']

    # Get site status
    site_status = df_sites[df_sites['site_id'] == site_id]['status'].values[0]

    for date in date_range:
        # If site is in care & maintenance, equipment is idle
        if site_status == 'Care & Maintenance':
            equipment_records.append({
                'date': date.strftime('%Y-%m-%d'),
                'site_id': site_id,
                'equipment_id': equipment_id,
                'equipment_type': equipment_type,
                'model': model,
                'autonomous': is_autonomous,
                'available_hours': 0.0,
                'operating_hours': 0.0,
                'idle_hours': 24.0,
                'maintenance_hours': 0.0,
                'breakdown_hours': 0.0,
                'availability_pct': 0.0,
                'utilization_pct': 0.0,
                'oee_pct': 0.0,
                'fuel_consumed_liters': 0,
                'maintenance_cost_aud': 0
            })
            continue

        # Random breakdown events (rare)
        has_breakdown = np.random.random() < 0.02  # 2% chance per day
        breakdown_hours = np.random.uniform(4, 12) if has_breakdown else 0

        # Scheduled maintenance (every ~15 days)
        has_maintenance = (date.day % 15 == 0)
        maintenance_hours = np.random.uniform(2, 6) if has_maintenance else 0

        # Total downtime
        total_downtime = breakdown_hours + maintenance_hours
        available_hours = 24 - total_downtime

        # Operating hours (% of available time)
        util_variance = np.random.normal(1.0, 0.1)
        actual_util = (target_util / 100) * util_variance
        actual_util = np.clip(actual_util, 0.5, 1.0)
        operating_hours = available_hours * actual_util
        idle_hours = available_hours - operating_hours

        # Calculate metrics
        availability_pct = (available_hours / 24) * 100
        utilization_pct = (operating_hours / available_hours * 100) if available_hours > 0 else 0
        oee_pct = (operating_hours / 24) * 100  # Simplified OEE

        # Fuel consumption (varies by equipment type)
        if equipment_type == 'Haul Truck':
            fuel_per_hour = 180  # liters/hour
        elif equipment_type == 'Excavator':
            fuel_per_hour = 400
        elif equipment_type == 'Dozer':
            fuel_per_hour = 120
        elif equipment_type == 'LHD Loader':
            fuel_per_hour = 80
        elif equipment_type == 'Jumbo Drill':
            fuel_per_hour = 60
        else:
            fuel_per_hour = 100

        fuel_consumed = int(operating_hours * fuel_per_hour)

        # Maintenance costs
        if has_maintenance:
            maintenance_cost = np.random.uniform(5000, 15000)
        elif has_breakdown:
            maintenance_cost = np.random.uniform(20000, 80000)
        else:
            maintenance_cost = np.random.uniform(500, 2000)  # Daily wear

        equipment_records.append({
            'date': date.strftime('%Y-%m-%d'),
            'site_id': site_id,
            'equipment_id': equipment_id,
            'equipment_type': equipment_type,
            'model': model,
            'autonomous': is_autonomous,
            'available_hours': round(available_hours, 1),
            'operating_hours': round(operating_hours, 1),
            'idle_hours': round(idle_hours, 1),
            'maintenance_hours': round(maintenance_hours, 1),
            'breakdown_hours': round(breakdown_hours, 1),
            'availability_pct': round(availability_pct, 1),
            'utilization_pct': round(utilization_pct, 1),
            'oee_pct': round(oee_pct, 1),
            'fuel_consumed_liters': fuel_consumed,
            'maintenance_cost_aud': round(maintenance_cost, 2)
        })

df_equipment = pd.DataFrame(equipment_records)

print(f"✓ Created equipment performance data: {len(df_equipment)} records")
print(f"  Equipment units tracked: {len(equipment_fleet)}")
print(f"  Total operating hours: {df_equipment['operating_hours'].sum():,.0f}")

# =====================================================
# DATASET 4: SAFETY INCIDENTS DATA
# =====================================================

print("\n[4/5] Generating Safety Incidents Data...")

# Safety incident types and their frequency (per year per site)
incident_types = [
    {'type': 'Lost Time Injury (LTI)', 'frequency': 2, 'severity': 'High'},
    {'type': 'Medical Treatment Injury (MTI)', 'frequency': 8, 'severity': 'Medium'},
    {'type': 'First Aid Injury (FAI)', 'frequency': 20, 'severity': 'Low'},
    {'type': 'Potential Significant Incident (PSI)', 'frequency': 15, 'severity': 'Medium'},
    {'type': 'Near Miss', 'frequency': 40, 'severity': 'Low'},
    {'type': 'Environmental Incident', 'frequency': 5, 'severity': 'Medium'},
]

safety_records = []
incident_id_counter = 1

for site in df_sites.itertuples():
    if site.status == 'Care & Maintenance':
        continue  # Minimal incidents for closed sites

    site_id = site.site_id
    site_name = site.site_name
    workforce = site.workforce_count

    # Generate incidents throughout the year
    for incident_type in incident_types:
        num_incidents = incident_type['frequency']

        # Randomly distribute incidents across the year
        incident_dates = sorted(random.sample(list(date_range), min(num_incidents, 365)))

        for inc_date in incident_dates:
            # Incident details
            incident_description = {
                'Lost Time Injury (LTI)': ['Fall from height', 'Crush injury', 'Struck by equipment', 'Back strain'],
                'Medical Treatment Injury (MTI)': ['Laceration', 'Minor burn', 'Sprain', 'Contusion'],
                'First Aid Injury (FAI)': ['Minor cut', 'Scrape', 'Splinter', 'Minor bruise'],
                'Potential Significant Incident (PSI)': ['Near miss vehicle collision', 'Geotechnical movement', 'Equipment malfunction', 'Uncontrolled energy release'],
                'Near Miss': ['Pedestrian near vehicle', 'Falling object', 'Slip/trip hazard', 'Equipment proximity'],
                'Environmental Incident': ['Minor fuel spill', 'Dust exceedance', 'Water quality issue', 'Waste handling'],
            }

            description = random.choice(incident_description[incident_type['type']])

            # Days lost (only for LTI)
            days_lost = random.randint(5, 30) if incident_type['type'] == 'Lost Time Injury (LTI)' else 0

            # Root cause
            root_causes = ['Human Error', 'Equipment Failure', 'Procedural Gap', 'Environmental Factor', 'Training Gap']
            root_cause = random.choice(root_causes)

            # Investigation status
            status_options = ['Closed', 'Closed', 'Closed', 'Under Investigation']  # Most closed
            investigation_status = random.choice(status_options)

            safety_records.append({
                'incident_id': f'INC-{incident_id_counter:05d}',
                'date': inc_date.strftime('%Y-%m-%d'),
                'site_id': site_id,
                'site_name': site_name,
                'incident_type': incident_type['type'],
                'severity': incident_type['severity'],
                'description': description,
                'days_lost': days_lost,
                'root_cause': root_cause,
                'investigation_status': investigation_status,
                'workforce_size': workforce
            })

            incident_id_counter += 1

df_safety = pd.DataFrame(safety_records)

# Calculate TRIFR and LTIFR (per million hours worked)
# Assume 12-hour shifts, 365 days, workforce size
def calculate_safety_metrics(df_sites, df_safety):
    metrics = []
    for site in df_sites.itertuples():
        if site.status == 'Care & Maintenance':
            continue

        site_id = site.site_id
        workforce = site.workforce_count

        # Total hours worked (workforce × 12 hours × 365 days)
        total_hours = workforce * 12 * 365

        # Count incidents
        site_incidents = df_safety[df_safety['site_id'] == site_id]
        lti_count = len(site_incidents[site_incidents['incident_type'] == 'Lost Time Injury (LTI)'])
        mti_count = len(site_incidents[site_incidents['incident_type'] == 'Medical Treatment Injury (MTI)'])
        total_recordable = lti_count + mti_count

        # Calculate rates (per million hours)
        trifr = (total_recordable / total_hours) * 1_000_000
        ltifr = (lti_count / total_hours) * 1_000_000

        metrics.append({
            'site_id': site_id,
            'site_name': site.site_name,
            'total_hours_worked': total_hours,
            'lti_count': lti_count,
            'mti_count': mti_count,
            'trifr': round(trifr, 2),
            'ltifr': round(ltifr, 2)
        })

    return pd.DataFrame(metrics)

df_safety_metrics = calculate_safety_metrics(df_sites, df_safety)

print(f"✓ Created safety incidents data: {len(df_safety)} incidents")
print("\nSafety Performance by Site:")
print(df_safety_metrics.to_string(index=False))

# =====================================================
# DATASET 5: DAILY OPERATING COSTS
# =====================================================

print("\n[5/5] Generating Daily Operating Costs Data...")

cost_records = []

for site in df_sites.itertuples():
    site_id = site.site_id
    site_name = site.site_name
    commodity = site.commodity
    status = site.status
    cost_target = site.cost_per_unit_target
    workforce = site.workforce_count

    for date in date_range:
        # Get production for this date
        prod_data = df_production[(df_production['site_id'] == site_id) & (df_production['date'] == date.strftime('%Y-%m-%d'))]

        if len(prod_data) == 0:
            continue

        tonnes_mined = prod_data['tonnes_mined'].values[0]
        metal_produced = prod_data['metal_produced'].values[0]

        # If care & maintenance
        if status == 'Care & Maintenance':
            cost_records.append({
                'date': date.strftime('%Y-%m-%d'),
                'site_id': site_id,
                'site_name': site_name,
                'commodity': commodity,
                'labor_cost_aud': 5000,  # Skeleton crew
                'equipment_cost_aud': 0,
                'fuel_cost_aud': 0,
                'maintenance_cost_aud': 2000,
                'consumables_cost_aud': 0,
                'utilities_cost_aud': 1000,
                'admin_cost_aud': 500,
                'total_cost_aud': 8500,
                'tonnes_produced': 0,
                'cost_per_unit': 0
            })
            continue

        # Cost breakdown (realistic percentages from research)
        # Base daily costs
        if commodity == 'Iron Ore':
            base_daily_cost = 137000 * 27  # Daily target × cost per tonne
        elif commodity == 'Gold':
            base_daily_cost = (35600 / 31.1) * 2500  # Tonnes to oz × cost per oz
        elif commodity == 'Lithium':
            base_daily_cost = 1920 * 640
        elif commodity == 'Copper':
            base_daily_cost = 55 * 2.2 * 1.85 * 1000  # Tonnes to lb × cost per lb
        else:
            base_daily_cost = 100000

        # Adjust for actual production
        production_ratio = tonnes_mined / site.daily_target_tonnes if site.daily_target_tonnes > 0 else 0
        daily_cost = base_daily_cost * production_ratio * np.random.uniform(0.95, 1.05)

        # Breakdown
        labor_cost = daily_cost * 0.40
        equipment_cost = daily_cost * 0.28
        fuel_cost = daily_cost * 0.18
        maintenance_cost = daily_cost * 0.08
        consumables_cost = daily_cost * 0.04
        utilities_cost = daily_cost * 0.01
        admin_cost = daily_cost * 0.01

        total_cost = labor_cost + equipment_cost + fuel_cost + maintenance_cost + consumables_cost + utilities_cost + admin_cost

        # Cost per unit
        if commodity == 'Gold':
            cost_per_unit = (total_cost / metal_produced) if metal_produced > 0 else 0  # AUD per oz
        elif commodity == 'Iron Ore':
            cost_per_unit = (total_cost / tonnes_mined) if tonnes_mined > 0 else 0  # AUD per tonne
        elif commodity == 'Lithium':
            cost_per_unit = (total_cost / metal_produced) if metal_produced > 0 else 0  # AUD per tonne concentrate
        elif commodity == 'Copper':
            cost_per_unit = (total_cost / (metal_produced * 1000 * 2.2)) if metal_produced > 0 else 0  # AUD per lb
        else:
            cost_per_unit = 0

        cost_records.append({
            'date': date.strftime('%Y-%m-%d'),
            'site_id': site_id,
            'site_name': site_name,
            'commodity': commodity,
            'labor_cost_aud': round(labor_cost, 2),
            'equipment_cost_aud': round(equipment_cost, 2),
            'fuel_cost_aud': round(fuel_cost, 2),
            'maintenance_cost_aud': round(maintenance_cost, 2),
            'consumables_cost_aud': round(consumables_cost, 2),
            'utilities_cost_aud': round(utilities_cost, 2),
            'admin_cost_aud': round(admin_cost, 2),
            'total_cost_aud': round(total_cost, 2),
            'tonnes_produced': tonnes_mined,
            'cost_per_unit': round(cost_per_unit, 2)
        })

df_costs = pd.DataFrame(cost_records)

print(f"✓ Created daily operating costs data: {len(df_costs)} records")
print(f"  Total operating costs: ${df_costs['total_cost_aud'].sum():,.0f} AUD")

# =====================================================
# DATA QUALITY SUMMARY
# =====================================================

print("\n" + "=" * 60)
print("DATA GENERATION COMPLETE!")
print("=" * 60)

print("\n📊 DATASET SUMMARY:")
print(f"  1. Mine Sites: {len(df_sites)} sites")
print(f"  2. Daily Production: {len(df_production)} records ({len(date_range)} days × {len(df_sites)} sites)")
print(f"  3. Equipment Performance: {len(df_equipment)} records ({len(date_range)} days × {len(equipment_fleet)} units)")
print(f"  4. Safety Incidents: {len(df_safety)} incidents")
print(f"  5. Daily Costs: {len(df_costs)} records")

print("\n💾 SAVING CSV FILES...")

# Save all datasets
df_sites.to_csv('mine_sites.csv', index=False)
print("  ✓ mine_sites.csv")

df_production.to_csv('daily_production.csv', index=False)
print("  ✓ daily_production.csv")

df_equipment.to_csv('equipment_performance.csv', index=False)
print("  ✓ equipment_performance.csv")

df_safety.to_csv('safety_incidents.csv', index=False)
print("  ✓ safety_incidents.csv")

df_costs.to_csv('daily_costs.csv', index=False)
print("  ✓ daily_costs.csv")

df_safety_metrics.to_csv('safety_metrics_summary.csv', index=False)
print("  ✓ safety_metrics_summary.csv (bonus file)")

print("\n" + "=" * 60)
print("✅ ALL FILES READY FOR DOWNLOAD!")
print("=" * 60)

print("\n📥 NEXT STEPS:")
print("  1. Download all CSV files using the folder icon (left sidebar)")
print("  2. Create a 'data/' folder in your project")
print("  3. Move all CSVs into the data folder")
print("  4. Ready for SQLite database creation!")

print("\n📈 QUICK DATA PREVIEW:")
print("\n--- Mine Sites ---")
print(df_sites[['site_id', 'site_name', 'commodity', 'status', 'daily_target_tonnes']].head())

print("\n--- Production Sample (First 5 Days, Iron Ridge) ---")
print(df_production[df_production['site_id'] == 'IRON001'].head()[['date', 'site_name', 'tonnes_mined', 'plan_adherence_pct']])

print("\n--- Equipment Sample (Iron Ridge Truck) ---")
print(df_equipment[df_equipment['equipment_id'] == 'TRK-IR-001'].head()[['date', 'equipment_id', 'availability_pct', 'utilization_pct', 'oee_pct']])

print("\n--- Safety Incidents by Type ---")
print(df_safety['incident_type'].value_counts())

print("\n--- Cost Summary by Site ---")
cost_summary = df_costs.groupby(['site_name', 'commodity']).agg({
    'total_cost_aud': 'sum',
    'cost_per_unit': 'mean'
}).reset_index()
cost_summary['total_cost_aud'] = cost_summary['total_cost_aud'].apply(lambda x: f"${x:,.0f}")
cost_summary['cost_per_unit'] = cost_summary['cost_per_unit'].apply(lambda x: f"${x:.2f}")
print(cost_summary.to_string(index=False))

print("\n" + "=" * 60)
print("🎉 DATA GENERATION SCRIPT COMPLETE!")
print("Based on real 2025 WA mining industry data")
print("=" * 60)

WA MINING OPERATIONS DATA GENERATOR
Generating realistic 2025 mining data for 5 WA mine sites

[1/5] Generating Mine Sites Master Data...
✓ Created mine sites data: 5 sites
site_id          site_name commodity             status
IRON001    Iron Ridge Mine  Iron Ore        Operational
GOLD001 Golden Valley Mine      Gold        Operational
LITH001 Lithium Creek Mine   Lithium        Operational
COPP001  Copper Hills Mine    Copper        Operational
NICK001  Nickel Point Mine    Nickel Care & Maintenance

[2/5] Generating Daily Production Data (365 days)...
✓ Created daily production data: 1825 records
  Date range: 2025-01-01 to 2025-12-31
  Total tonnes mined: 59,706,811

[3/5] Generating Equipment Performance Data...
✓ Created equipment performance data: 8030 records
  Equipment units tracked: 22
  Total operating hours: 146,822

[4/5] Generating Safety Incidents Data...
✓ Created safety incidents data: 360 incidents

Safety Performance by Site:
site_id          site_name  total_hour

In [ ]:
from google.colab import files
import shutil

# Create a zip file with all CSVs
!zip mining_data.zip *.csv

# Download the zip
files.download('mining_data.zip')

print("✓ All files zipped and ready for download!")

  adding: daily_costs.csv (deflated 73%)
  adding: daily_production.csv (deflated 83%)
  adding: equipment_performance.csv (deflated 87%)
  adding: mine_sites.csv (deflated 41%)
  adding: safety_incidents.csv (deflated 91%)
  adding: safety_metrics_summary.csv (deflated 27%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ All files zipped and ready for download!


In [ ]:
# =====================================================
# MINING OPERATIONS DATABASE CREATOR
# Creates SQLite database from CSV files
# =====================================================

import sqlite3
import pandas as pd
from datetime import datetime

print("=" * 60)
print("MINING OPERATIONS DATABASE CREATOR")
print("Creating SQLite database from CSV files...")
print("=" * 60)

# =====================================================
# STEP 1: LOAD ALL CSV FILES
# =====================================================

print("\n[1/3] Loading CSV files...")

try:
    df_sites = pd.read_csv('mine_sites.csv')
    print(f"  ✓ Loaded mine_sites.csv ({len(df_sites)} records)")

    df_production = pd.read_csv('daily_production.csv')
    print(f"  ✓ Loaded daily_production.csv ({len(df_production)} records)")

    df_equipment = pd.read_csv('equipment_performance.csv')
    print(f"  ✓ Loaded equipment_performance.csv ({len(df_equipment)} records)")

    df_safety = pd.read_csv('safety_incidents.csv')
    print(f"  ✓ Loaded safety_incidents.csv ({len(df_safety)} records)")

    df_costs = pd.read_csv('daily_costs.csv')
    print(f"  ✓ Loaded daily_costs.csv ({len(df_costs)} records)")

    df_safety_metrics = pd.read_csv('safety_metrics_summary.csv')
    print(f"  ✓ Loaded safety_metrics_summary.csv ({len(df_safety_metrics)} records)")

except FileNotFoundError as e:
    print(f"\n❌ ERROR: Could not find CSV files!")
    print(f"   Make sure all CSV files are in the same folder as this script.")
    print(f"   Missing file: {e.filename}")
    exit()

# =====================================================
# STEP 2: CREATE SQLite DATABASE
# =====================================================

print("\n[2/3] Creating SQLite database...")

# Create database connection
db_name = 'mining_operations.db'
conn = sqlite3.connect(db_name)

print(f"  ✓ Created database: {db_name}")

# =====================================================
# STEP 3: CREATE TABLES AND LOAD DATA
# =====================================================

print("\n[3/3] Creating tables and loading data...")

# Table 1: Mine Sites (Master Data)
df_sites.to_sql('mine_sites', conn, if_exists='replace', index=False)
print(f"  ✓ Created table: mine_sites ({len(df_sites)} records)")

# Table 2: Daily Production
df_production.to_sql('daily_production', conn, if_exists='replace', index=False)
print(f"  ✓ Created table: daily_production ({len(df_production)} records)")

# Table 3: Equipment Performance
df_equipment.to_sql('equipment_performance', conn, if_exists='replace', index=False)
print(f"  ✓ Created table: equipment_performance ({len(df_equipment)} records)")

# Table 4: Safety Incidents
df_safety.to_sql('safety_incidents', conn, if_exists='replace', index=False)
print(f"  ✓ Created table: safety_incidents ({len(df_safety)} records)")

# Table 5: Daily Costs
df_costs.to_sql('daily_costs', conn, if_exists='replace', index=False)
print(f"  ✓ Created table: daily_costs ({len(df_costs)} records)")

# Table 6: Safety Metrics Summary
df_safety_metrics.to_sql('safety_metrics_summary', conn, if_exists='replace', index=False)
print(f"  ✓ Created table: safety_metrics_summary ({len(df_safety_metrics)} records)")

# =====================================================
# CREATE INDEXES FOR PERFORMANCE
# =====================================================

print("\n[BONUS] Creating indexes for query performance...")

cursor = conn.cursor()

# Indexes on frequently queried columns
indexes = [
    "CREATE INDEX IF NOT EXISTS idx_production_site_date ON daily_production(site_id, date);",
    "CREATE INDEX IF NOT EXISTS idx_production_date ON daily_production(date);",
    "CREATE INDEX IF NOT EXISTS idx_equipment_site_date ON equipment_performance(site_id, date);",
    "CREATE INDEX IF NOT EXISTS idx_equipment_id ON equipment_performance(equipment_id);",
    "CREATE INDEX IF NOT EXISTS idx_safety_site_date ON safety_incidents(site_id, date);",
    "CREATE INDEX IF NOT EXISTS idx_costs_site_date ON daily_costs(site_id, date);",
]

for idx_sql in indexes:
    cursor.execute(idx_sql)
    print(f"  ✓ Created index")

conn.commit()

# =====================================================
# VERIFY DATABASE STRUCTURE
# =====================================================

print("\n" + "=" * 60)
print("DATABASE STRUCTURE:")
print("=" * 60)

# Get table list
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"  📊 {table_name}: {count:,} records")

# =====================================================
# TEST QUERIES
# =====================================================

print("\n" + "=" * 60)
print("RUNNING TEST QUERIES:")
print("=" * 60)

# Test Query 1: Total production by site
print("\n--- Total Production by Site (2025) ---")
query1 = """
SELECT
    site_name,
    commodity,
    SUM(tonnes_mined) as total_tonnes,
    ROUND(AVG(plan_adherence_pct), 1) as avg_plan_adherence
FROM daily_production
WHERE status = 'Operational'
GROUP BY site_name, commodity
ORDER BY total_tonnes DESC;
"""
result1 = pd.read_sql_query(query1, conn)
print(result1.to_string(index=False))

# Test Query 2: Equipment availability by type
print("\n--- Equipment Availability by Type ---")
query2 = """
SELECT
    equipment_type,
    COUNT(DISTINCT equipment_id) as unit_count,
    ROUND(AVG(availability_pct), 1) as avg_availability,
    ROUND(AVG(utilization_pct), 1) as avg_utilization,
    ROUND(AVG(oee_pct), 1) as avg_oee
FROM equipment_performance
GROUP BY equipment_type
ORDER BY avg_oee DESC;
"""
result2 = pd.read_sql_query(query2, conn)
print(result2.to_string(index=False))

# Test Query 3: Safety performance
print("\n--- Safety Performance by Site ---")
query3 = """
SELECT
    site_name,
    trifr,
    ltifr,
    lti_count,
    mti_count
FROM safety_metrics_summary
ORDER BY trifr;
"""
result3 = pd.read_sql_query(query3, conn)
print(result3.to_string(index=False))

# Test Query 4: Cost efficiency by site
print("\n--- Cost Efficiency by Site ---")
query4 = """
SELECT
    site_name,
    commodity,
    ROUND(AVG(cost_per_unit), 2) as avg_cost_per_unit,
    ROUND(SUM(total_cost_aud) / 1000000, 2) as total_cost_millions_aud
FROM daily_costs
WHERE commodity != 'Nickel'  -- Exclude Care & Maintenance site
GROUP BY site_name, commodity
ORDER BY total_cost_millions_aud DESC;
"""
result4 = pd.read_sql_query(query4, conn)
print(result4.to_string(index=False))

# Close connection
conn.close()

# =====================================================
# DOWNLOAD DATABASE FILE
# =====================================================

print("\n" + "=" * 60)
print("✅ DATABASE CREATION COMPLETE!")
print("=" * 60)

print(f"\n📦 Database file created: {db_name}")
print(f"   Size: {pd.io.common.get_handle(db_name, 'rb').handle.seek(0, 2) / 1024 / 1024:.2f} MB")

MINING OPERATIONS DATABASE CREATOR
Creating SQLite database from CSV files...

[1/3] Loading CSV files...
  ✓ Loaded mine_sites.csv (5 records)
  ✓ Loaded daily_production.csv (1825 records)
  ✓ Loaded equipment_performance.csv (8030 records)
  ✓ Loaded safety_incidents.csv (360 records)
  ✓ Loaded daily_costs.csv (1825 records)
  ✓ Loaded safety_metrics_summary.csv (4 records)

[2/3] Creating SQLite database...
  ✓ Created database: mining_operations.db

[3/3] Creating tables and loading data...
  ✓ Created table: mine_sites (5 records)
  ✓ Created table: daily_production (1825 records)
  ✓ Created table: equipment_performance (8030 records)
  ✓ Created table: safety_incidents (360 records)
  ✓ Created table: daily_costs (1825 records)
  ✓ Created table: safety_metrics_summary (4 records)

[BONUS] Creating indexes for query performance...
  ✓ Created index
  ✓ Created index
  ✓ Created index
  ✓ Created index
  ✓ Created index
  ✓ Created index

DATABASE STRUCTURE:
  📊 mine_sites: 5 r